In [ ]:
import pickle
import pandas as pd

In [ ]:
uniprot = pd.read_csv("../uniprot.tsv.gz", sep='\t').set_index("Entry")[['Protein families', "Cross-reference (Pfam)"]]
prots = pd.read_pickle("results/prepare_proteins/protein_data_whole_label_none.pkl")
prots.index = prots.index.to_series().apply(lambda x:x.split("-")[1])
prots = prots.join(uniprot)

In [ ]:
prots = prots[prots['Cross-reference (Pfam)'].notna()].reset_index().drop_duplicates("index").set_index("index")
prots.drop("Protein families", axis=1, inplace=True)

In [ ]:
prots['fam'] = prots['Cross-reference (Pfam)']
prots['nnodes'] = prots.data.apply(lambda x: x['x'].size(0))
prots = prots[prots['nnodes'].between(150, 1000)]

In [ ]:
import plotly.express as px
px.histogram(prots['nnodes'])

In [ ]:
def add_fam(row):
    data = row['data']
    if "index_mapping" in data:
        del data['index_mapping']
    data['fam'] = row['fam']
    return data

prots['data'] = prots.apply(add_fam, axis=1)

In [ ]:
prots[['data']].to_pickle("/scratch/SCRATCH_NVME/ilya/pretrain_data/pfam_1000.pkl")